In [1]:
import pandas as pd
import jieba

import jieba.posseg as pseg
jieba.load_userdict('userdict/symptom.txt')
jieba.load_userdict('userdict/time.txt')
jieba.load_userdict('userdict/position.txt')
jieba.load_userdict('userdict/others.txt')

import re
from collections import defaultdict
import matplotlib.pyplot as plt

import time as timer
start_time = timer.time()



kcom = pd.read_csv('Kcom_dr568410605emgcase.csv', lineterminator='\n')
kcom['Kcom'] = kcom['Kcom'].fillna("並無主訴")

print(len(kcom))
kcom.head()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/tv/yy9kh1h15dsfg0vcy7vbgtmw0000gn/T/jieba.cache
Loading model cost 1.101 seconds.
Prefix dict has been built succesfully.


241822


,Pno,Caseno,Kcom
0,15929545,"5,621",今天下午開始雙腳抽筋 現求治
1,15941825,"5,011",剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
2,15965603,"6,001",據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
3,15966387,"6,801",昨天開始頭暈不適 現未改善
4,15968549,"5,441",昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中


In [2]:
p = pd.DataFrame(columns=['Pno','Caseno'])
kcom_out = pd.concat([pd.DataFrame(columns=['Kcom_'+str(i),'Kcom_du'+str(i)]) for i in range(1,11)], axis=1)
kcom_out = pd.concat([p, kcom_out], axis=1)

kcom_out

,Pno,Caseno,Kcom_1,Kcom_du1,Kcom_2,Kcom_du2,Kcom_3,Kcom_du3,Kcom_4,Kcom_du4,...,Kcom_6,Kcom_du6,Kcom_7,Kcom_du7,Kcom_8,Kcom_du8,Kcom_9,Kcom_du9,Kcom_10,Kcom_du10


In [3]:
max_length = 0

with open('output.txt', 'a') as the_file:

    for i in range(len(kcom)):
        split = kcom['Kcom'].loc[i].replace("有","")
        split = split.split()


        t_now = 0
        pre_time = list()
        event_w = list()
        event_now = list()

        for s in split: 
            t_w = list()
            sym_w = list()

            last_t = 0
            last_n = 0
            last_sym = 0
            t = 0
            sym = 0


            line = pseg.cut(s,HMM=False)
            for k,f in line:
                #print('%s %s' % (k,f), end=' ')

                if f == 't':
                    last_n = 0
                    last_sym = 0
                    if t_now == 0:
                        if sym == 1:
                            event = dict()
                            event['time'] = t_w
                            event['sym'] = sym_w
                            event_w.append(event)
                            t_w = list()
                            sym_w = list()
                            sym = 0
                            t_w.append(k)
                            last_t = 1
                            t = 1  
                        elif last_t == 1 and t == 1:
                            t_w.append(k)
                            last_t = 1
                            t = 1
                        elif t == 0:
                            t_w.append(k)
                            last_t = 1
                            t = 1

                elif f == 'tnow' or f == 'tg':
                    last_n = 0
                    last_t = 0
                    last_sym = 0
                    t_now = 1
                    if sym == 1:
                        event = dict()
                        event['time'] = t_w
                        event['sym'] = sym_w
                        event_w.append(event)
                        t_w = list()
                        sym_w = list()
                        sym = 0


                elif f == 'sym':
                    if last_n == 1:
                        sym_w.append(n+k)
                    else:
                        sym_w.append(k)

                    last_n = 0
                    last_t = 0
                    last_sym = 1
                    sym = 1

                elif f == 'f':
                    last_n = 1
                    last_sym = 0
                    n = k
                elif f == 'pos':
                    if last_n == 1:
                        n = n+k
                    else: 
                        n = k
                        
                    last_n = 1
                    last_sym = 0
                elif f == 'm' and last_sym == 1:
                    last_t = 0
                    last_n = 0
                    last_sym = 0
                    if len(t_w) == 0:
                        t_w.append(k)
                        last_t = 1
                
                else:
                    last_t = 0
                    last_n = 0

            if len(sym_w) > 0:
                if t_now == 1:
                    event_now += sym_w
                else:
                    event = dict()
                    if len(t_w) == 0:
                        event['time'] = pre_time
                    else:
                        event['time'] = t_w 
                    event['sym'] = sym_w
                    event_w.append(event)
            elif len(t_w) > 0:
                    pre_time = t_w

        total = list()
        
        time = ''
        for e in event_w:
            event = dict()
            #print(e)
            if len(e['time']) > 0:
                time = ''.join(e['time'])
            event['time'] = time
            event['sym'] = e['sym']

            total.append(event)
        
        '''
        for idx, e in reversed(list(enumerate(total))):
            if idx != 0:
                if e['time'] == total[idx-1]['time']:
                    #print(e)
                    total[idx-1]['sym'] += e['sym']
                    del total[idx]
        '''
        
    
        
        if len(event_now) > 0:
            #total.append({'time': '現在', 'sym': event_now})
            for e_now in event_now:
                total.append({'time': '現在', 'sym': [e_now]})

        '''
        if len(total) > max_length:
            max_length = len(total)
        '''
        
        
        
        p = pd.DataFrame(columns=['Pno','Caseno'])
        temp = pd.concat([pd.DataFrame(columns=['Kcom_'+str(t),'Kcom_du'+str(t)]) for t in range(1,11)], axis=1)
        temp = pd.concat([p, temp], axis=1)
        
        temp['Pno'] = [kcom['Pno'].loc[i]]
        temp['Caseno'] = [kcom['Caseno'].loc[i]]
        
        t_idx = 1 
        for period in total:
            #print(period)
            
            if len(period['sym']) > 1:
                for s in period['sym']:
                    temp['Kcom_'+ str(t_idx)] = [s]
                    temp['Kcom_du'+ str(t_idx)] = period['time']
                    t_idx += 1
            else:
                temp['Kcom_'+ str(t_idx)] = period['sym']
                temp['Kcom_du'+ str(t_idx)] = period['time']
                t_idx += 1
        
        kcom_out = kcom_out.append(temp)
        
        
        #print('%d. %s' % (i+1,kcom['Kcom'].loc[i]))
        #the_file.write('%d. %s\n' % (i+1,kcom['Kcom'].loc[i]))
        #for period in total:
            #print('時間： %s' % period['time'])
            #print('症狀： %s' % ','.join(period['sym']))
            #the_file.write('時間： %s\n' % period['time'])
            #the_file.write('症狀： %s\n' % ','.join(period['sym']))
        #print('\n')
        #the_file.write('\n')
    #print(max_length)
    #kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")
    

In [4]:
kcom_out.to_csv('kcom_output_0524.csv', index=False,encoding="utf_8_sig")
#kcom_out.head()

In [5]:
#import time
print("--- %s seconds ---" % (timer.time() - start_time))

--- 18844.9210190773 seconds ---


In [6]:
#kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")